In [1]:
# 1) Conectar o Google Colab ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) Bibliotecas
import pandas as pd
import numpy as np
import os

# 3) Caminho do parquet original
caminho_application = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/application_processed.parquet"

# 4) Ler parquet
df_app = pd.read_parquet(caminho_application)

# 5) Seleção das colunas desejadas
colunas_desejadas = [
    "job_id",
    "titulo_profissional",
    "endereco",
    "sexo",
    "pcd",
    "nivel_profissional",
    "nivel_academico",
    "nivel_ingles",
    "nivel_espanhol",
    "cv_pt"
]

df_app = df_app[colunas_desejadas].copy()

# 6) Dicionários de normalização
GLOSSARIO = {
    "senioridade_group": {
        "Aprendiz": "Estagiário",
        "Trainee": "Estagiário",
        "Assistente": "Assistente",
        "Auxiliar": "Assistente",
        "Técnico de Nível Médio": "Assistente",
        "Júnior": "Analista Júnior",
        "Pleno": "Analista Pleno",
        "Analista": "Analista Pleno",
        "Sênior": "Analista Sênior",
        "Especialista": "Especialista",
        "Líder": "Líder",
        "Supervisor": "Supervisor",
        "Coordenador": "Coordenador",
        "Gerente": "Gerente",
    },
    "senioridade_lvl": {
        "Estagiário": 1,
        "Assistente": 2,
        "Analista Júnior": 3,
        "Analista Pleno": 4,
        "Analista Sênior": 5,
        "Especialista": 6,
        "Líder": 7,
        "Supervisor": 8,
        "Coordenador": 9,
        "Gerente": 10,
    },
    "idioma_nvl": {
        "Nenhum": 0,
        "Básico": 1,
        "Técnico": 2,
        "Intermediário": 3,
        "Avançado": 4,
        "Fluente": 5,
    },
    "nivel_academico_lvl": {
        "Ensino Fundamental Completo": 1,
        "Ensino Médio Incompleto": 2,
        "Ensino Médio Completo": 3,
        "Ensino Técnico Incompleto": 4,
        "Ensino Técnico Cursando": 5,
        "Ensino Técnico Completo": 6,
        "Ensino Superior Incompleto": 7,
        "Ensino Superior Cursando": 8,
        "Ensino Superior Completo": 9,
        "Pós Graduação Incompleto": 10,
        "Pós Graduação Cursando": 11,
        "Pós Graduação Completo": 12,
        "Mestrado Cursando": 13,
        "Mestrado Completo": 14,
        "Doutorado Cursando": 15,
    },
}

# 7) Normalização de idiomas
for col in ["nivel_ingles", "nivel_espanhol"]:
    df_app[col] = (
        df_app[col]
        .astype("string")
        .str.strip()
        .replace(["", None], np.nan)
        .fillna("Nenhum")
    )
    df_app[f"{col}_nvl"] = df_app[col].map(GLOSSARIO["idioma_nvl"]).fillna(0).astype(int)

# 8) Normalização de senioridade
df_app["nivel_profissional"] = df_app["nivel_profissional"].astype("string").str.strip()
df_app["senioridade_group"] = (
    df_app["nivel_profissional"].map(GLOSSARIO["senioridade_group"]).fillna("None")
)
df_app["senioridade_lvl"] = (
    df_app["senioridade_group"].map(GLOSSARIO["senioridade_lvl"]).fillna(0).astype(int)
)

# 9) Normalização de nível acadêmico
df_app["nivel_academico"] = df_app["nivel_academico"].astype("string").str.strip()
df_app["nivel_academico_lvl"] = (
    df_app["nivel_academico"].map(GLOSSARIO["nivel_academico_lvl"]).fillna(0).astype(int)
)

# 10) Garantir inteiros finais
for col in ["nivel_ingles_nvl", "nivel_espanhol_nvl", "senioridade_lvl", "nivel_academico_lvl"]:
    if col in df_app.columns:
        df_app[col] = df_app[col].fillna(0).astype(int)

# 11) Prévia
print("Dimensão final:", df_app.shape)
display(df_app.head(10))

# 12) Exportar parquet
path_parquet = "/content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/talent_pool.parquet"
os.makedirs(os.path.dirname(path_parquet), exist_ok=True)
df_app.to_parquet(path_parquet, index=False)
print("Exportado para:", path_parquet)

Mounted at /content/drive
Dimensão final: (42482, 15)


,job_id,titulo_profissional,endereco,sexo,pcd,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,cv_pt,nivel_ingles_nvl,nivel_espanhol_nvl,senioridade_group,senioridade_lvl,nivel_academico_lvl
0,31000,,,None,None,,<NA>,Nenhum,Nenhum,assistente administrativo\n\n\nsantosbatista\n...,0,0,None,0,0
1,31001,Analista Administrativo,são paulo,Feminino,Não,,Ensino Superior Incompleto,Nenhum,Nenhum,formação acadêmica\nensino médio (2º grau) em ...,0,0,None,0,7
2,31002,Administrativo | Financeiro,são paulo,Feminino,Não,,Ensino Superior Completo,Intermediário,Básico,objetivo: área administrativa | financeira\n\n...,3,1,None,0,9
3,31003,Área administrativa,são paulo,Feminino,Não,,Ensino Superior Incompleto,Nenhum,Nenhum,formação\nensino médio completo\ninformática i...,0,0,None,0,7
4,31004,,,None,None,,<NA>,Nenhum,Nenhum,última atualização em 09/11/2021\n­ sp\n\nensi...,0,0,None,0,0
5,31005,Administrativa/logística e ou comercial,são paulo,Feminino,None,,Ensino Técnico Completo,Básico,Básico,"brasileira\n41 anos, casada\nobjetivo: adminis...",1,1,None,0,6
6,31006,Analista Administrativo Financeiro,são paulo,Masculino,None,,Ensino Superior Cursando,Nenhum,Intermediário,objetivo\nárea financeira / controladoria / fa...,0,3,None,0,8
7,31007,Desejo fazer parte do time da empresa oferecen...,minas gerais,Feminino,Não,,Ensino Superior Completo,Nenhum,Nenhum,belo horizonte/mg – cep: 30.626-710\nhabilitaç...,0,0,None,0,9
8,31008,,,None,None,,<NA>,Nenhum,Nenhum,última atualização em 04/11/2021\n\nresumo\nex...,0,0,None,0,0
9,31009,,,None,None,,<NA>,Nenhum,Nenhum,"nasceu em 28 de maio de 1990, 31 anos.\nfemini...",0,0,None,0,0


Exportado para: /content/drive/My Drive/Tech Challenge 5/Dados/silver/dados_processed/talent_pool.parquet
